In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import ShuffleSplit

In [2]:
df = pd.read_csv('data/DefaultCreditcardClients.csv')
df.rename(columns={'default payment next month':'default'}, inplace=True)
df.index = df.ID

In [3]:
# 1. Remove attributes that just arent useful for us
del df['ID']

In [4]:
#Create Lists for Analysis
continuous_features = ['LIMIT_BAL', 'BILL_AMT1', 'BILL_AMT2','BILL_AMT3',
                       'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
                       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5',
                       'PAY_AMT6']
ordinal_features = ['EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0','PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6','default']

In [5]:
#Convert datatypes
df[continuous_features] = df[continuous_features].astype(np.float64)
df[ordinal_features] = df[ordinal_features].astype(np.int64)

In [6]:
#Transformations

#convert any non-identified education categories to 'OTHER'
df['EDUCATION'] = df['EDUCATION'].replace(to_replace=(0,5,6),value=4)

#convert any non-identified marriage categories to 'OTHER'
df['MARRIAGE'] = df['MARRIAGE'].replace(to_replace=(0),value=3)

#Log transform continuous variables; as they each have a mostly exponential distribution
df["log_LIMIT_BAL"]=np.log(df.LIMIT_BAL)
df["log_PAY_AMT1"]=np.log(df.PAY_AMT1+1)
df["log_PAY_AMT2"]=np.log(df.PAY_AMT2+1)
df["log_PAY_AMT3"]=np.log(df.PAY_AMT3+1)
df["log_PAY_AMT4"]=np.log(df.PAY_AMT4+1)
df["log_PAY_AMT5"]=np.log(df.PAY_AMT5+1)
df["log_PAY_AMT6"]=np.log(df.PAY_AMT6+1)

In [7]:
#as determined in prio lab analysis
#high correlation between BILL Amount and Pay amount so ignore bill amount for now
dfsub = df

In [8]:
del dfsub['LIMIT_BAL']
del dfsub['BILL_AMT1']
del dfsub['BILL_AMT2']
del dfsub['BILL_AMT3']
del dfsub['BILL_AMT4']
del dfsub['BILL_AMT5']
del dfsub['BILL_AMT6']
del dfsub['PAY_AMT1']
del dfsub['PAY_AMT2']
del dfsub['PAY_AMT3']
del dfsub['PAY_AMT4']
del dfsub['PAY_AMT5']
del dfsub['PAY_AMT6']

In [9]:
dfsub[['EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0','PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6','default']].describe()

,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,default
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,1.842267,1.557267,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,-0.291100,0.221200
std,0.744494,0.521405,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,1.149988,0.415062
min,1.000000,1.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,0.000000
25%,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
50%,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,4.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,1.000000


In [10]:
from sklearn.model_selection import ShuffleSplit

#Isolate the "default" variable into y and keep everythign else in X to use for predictions.
if 'default' in dfsub:
    y = dfsub['default'].values
    del dfsub['default'] 
    X = dfsub.values

num_cv_iterations = 10
num_instances = len(y)
cv_object = ShuffleSplit(n_splits=num_cv_iterations,
                         test_size  = 0.2)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt

#Create logistic regression object
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object

iter_num=0
for train_indices, test_indices in cv_object.split(X,y): 
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    lr_clf.fit(X_train,y_train)
    y_hat = lr_clf.predict(X_test)
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    iter_num+=1

====Iteration 0  ====
accuracy 0.8041666666666667
confusion matrix
 [[4522  161]
 [1014  303]]
====Iteration 1  ====
accuracy 0.8096666666666666
confusion matrix
 [[4517  154]
 [ 988  341]]
====Iteration 2  ====
accuracy 0.8031666666666667
confusion matrix
 [[4474  149]
 [1032  345]]
====Iteration 3  ====
accuracy 0.8031666666666667
confusion matrix
 [[4502  155]
 [1026  317]]
====Iteration 4  ====
accuracy 0.8136666666666666
confusion matrix
 [[4547  158]
 [ 960  335]]
====Iteration 5  ====
accuracy 0.8088333333333333
confusion matrix
 [[4510  136]
 [1011  343]]
====Iteration 6  ====
accuracy 0.81
confusion matrix
 [[4530  158]
 [ 982  330]]
====Iteration 7  ====
accuracy 0.8068333333333333
confusion matrix
 [[4526  166]
 [ 993  315]]
====Iteration 8  ====
accuracy 0.8108333333333333
confusion matrix
 [[4536  162]
 [ 973  329]]
====Iteration 9  ====
accuracy 0.8123333333333334
confusion matrix
 [[4550  168]
 [ 958  324]]
